In [6]:
import os
import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision 
from torchvision import transforms
import numpy as np

In [7]:
if torch.cuda.is_available():
   DEVICE = torch.device("cuda")
else:
   DEVICE = torch.device("cpu")

print(DEVICE)

BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10


cpu


In [8]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=False)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

print("train_dataset size:", len(train_dataset),"\ntest_dataset size:", len(test_dataset))

train_dataset size: 60000 
test_dataset size: 10000


In [9]:
# Label Randomizer

randomLabel = torch.tensor(np.random.randint(0, 10, (len(train_dataset)),))
train_dataset.targets = randomLabel


In [10]:
# Data loader func
def train_loader(BATCHSIZE):
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size = BATCHSIZE, 
                                           shuffle=True)
    return train_loader


def test_loader(BATCHSIZE):
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=BATCHSIZE, 
                                          shuffle=False)
    return test_loader

In [11]:

def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


In [12]:
def objective(trial):

    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_load = train_loader(BATCHSIZE)
    test_load = test_loader(BATCHSIZE)

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_load):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_load):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(test_load.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [13]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("{}: {}".format(key, value))

[I 2022-02-11 10:39:18,819] A new study created in memory with name: no-name-f1df3058-6934-4f05-9e2c-05107ea52f8e
[I 2022-02-11 10:39:23,920] Trial 0 finished with value: 0.05 and parameters: {'n_layers': 1, 'n_units_l0': 114, 'dropout_l0': 0.438361804231545, 'optimizer': 'SGD', 'lr': 3.903638349992392e-05}. Best is trial 0 with value: 0.05.
[I 2022-02-11 10:39:30,374] Trial 1 finished with value: 0.15859375 and parameters: {'n_layers': 2, 'n_units_l0': 92, 'dropout_l0': 0.2787431024701959, 'n_units_l1': 11, 'dropout_l1': 0.2569989755972666, 'optimizer': 'Adam', 'lr': 0.0002618605864525328}. Best is trial 1 with value: 0.15859375.
[I 2022-02-11 10:39:35,790] Trial 2 finished with value: 0.06953125 and parameters: {'n_layers': 2, 'n_units_l0': 64, 'dropout_l0': 0.21604853955911038, 'n_units_l1': 49, 'dropout_l1': 0.2225280725994449, 'optimizer': 'Adam', 'lr': 0.00036061293401595205}. Best is trial 1 with value: 0.15859375.
[I 2022-02-11 10:39:41,107] Trial 3 finished with value: 0.11953

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  82
  Number of complete trials:  18
Best trial:
  Value:  0.1640625
  Params: 
n_layers: 1
n_units_l0: 96
dropout_l0: 0.3195759100801704
optimizer: Adam
lr: 6.729740042035753e-05
